In [4]:
def find_perpendicular_vector(v):
    # Unpack the vector components
    a, b, c = v

    # Check if the vector is not a zero vector
    if a == 0 and b == 0 and c == 0:
        raise ValueError("The input vector must not be the zero vector.")
    
    # If the vector is along the x-axis
    if a != 0 or b != 0:
        # Use a vector that is not parallel to v (e.g., along z-axis)
        v_perpendicular = np.array([-b, a, 0])
    else:
        # Use a vector that is not parallel to v (e.g., along y-axis)
        v_perpendicular = np.array([0, -c, b])
    
    # Normalize the perpendicular vector
    v_perpendicular = v_perpendicular / np.linalg.norm(v_perpendicular)
    
    return v_perpendicular

# Example usage:
V = (1, 2, 3)  # Replace with your vector (a, b, c)
V_perpendicular = find_perpendicular_vector(V)
print("Original Vector V:", V)
print("Perpendicular Vector V':", V_perpendicular)


Original Vector V: (1, 2, 3)
Perpendicular Vector V': [-0.89442719  0.4472136   0.        ]


In [5]:
def normalize_vector(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
        raise ValueError("The vector should not be zero")
    return v / norm

def quaternion_180_degree_rotation(v):
    # Normalize the vector V
    v_norm = normalize_vector(v)
    
    # The quaternion for 180-degree rotation around V
    qw = 0
    qx, qy, qz = v_norm
    
    return (qw, qx, qy, qz)

V = np.array([1, 2, 3])  
V0 = np.array([0, 1, 0])  # Initial vector (v0)
Q = quaternion_180_degree_rotation(V)

print("Quaternion Q = (QW, QX, QY, QZ):", Q)

Quaternion Q = (QW, QX, QY, QZ): (0, 0.2672612419124244, 0.5345224838248488, 0.8017837257372732)


In [6]:
def quaternion_to_rotation_matrix(q):
    QW, QX, QY, QZ = q
    
    # Compute the rotation matrix from the quaternion
    R = np.array([
        [1 - 2*QY**2 - 2*QZ**2, 2*QX*QY - 2*QZ*QW, 2*QX*QZ + 2*QY*QW],
        [2*QX*QY + 2*QZ*QW, 1 - 2*QX**2 - 2*QZ**2, 2*QY*QZ - 2*QX*QW],
        [2*QX*QZ - 2*QY*QW, 2*QY*QZ + 2*QX*QW, 1 - 2*QX**2 - 2*QY**2]
    ])
    
    return R

def rotate_vector_by_quaternion(v, q):
    # Get the rotation matrix from the quaternion
    R = quaternion_to_rotation_matrix(q)
    
    # Rotate the vector using the rotation matrix
    v_rotated = np.dot(R, v)
    
    return v_rotated

# Initial vector V = (0, 0, 1)
V = np.array([0,0,1])
Q =  (-0.29549739238222106, 0.268869009462429, 0.6639669823283616, 0.6320908582021806)  # Replace with your quaternion
V_rotated = rotate_vector_by_quaternion(V, Q)

print("Original Vector V:", V)
print("Rotated Vector V':", V_rotated)

Original Vector V: [0 0 1]
Rotated Vector V': [-0.05250174  0.9982751  -0.0262854 ]


In [7]:
def normalize_vector(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    return v / norm

def find_rotation_quaternion(v1, v2):
    # Normalize the vectors
    v1 = normalize_vector(v1)
    v2 = normalize_vector(v2)
    
    # Compute the dot product (cosine of the angle)
    dot_product = np.dot(v1, v2)
    
    # If the vectors are exactly opposite, special case handling
    if np.isclose(dot_product, -1.0):
        # Choose an arbitrary orthogonal vector for the axis of rotation
        orthogonal_vector = np.array([1, 0, 0]) if not np.allclose(v1, [1, 0, 0]) else np.array([0, 1, 0])
        # Compute the axis of rotation (cross product)
        axis = normalize_vector(np.cross(v1, orthogonal_vector))
        # 180 degrees rotation quaternion
        qw = 0
        qx, qy, qz = axis
    else:
        # Compute the axis of rotation (cross product)
        axis = np.cross(v1, v2)
        # Compute the angle of rotation (acos of the dot product)
        qw = np.sqrt((1.0 + dot_product) / 2.0)
        qx, qy, qz = axis * np.sqrt((1.0 - dot_product) / 2.0)
    
    return (qw, qx, qy, qz)

V_initial = np.array([0,1,0])  
V_target = np.array([ 0.32946418, -0.89981517,  0.28598426])  

Q = find_rotation_quaternion(V_initial, V_target)
print("Quaternion Q = (QW, QX, QY, QZ):", Q)


Quaternion Q = (QW, QX, QY, QZ): (0.22381384259430193, 0.2787292491219647, 0.0, -0.32110614585566283)


In [8]:
def quaternion_multiply(Q1, Q2):
    w1, x1, y1, z1 = Q1
    w2, x2, y2, z2 = Q2
    
    w = w2*w1 - x2*x1 - y2*y1 - z2*z1
    x = w2*x1 + x2*w1 + y2*z1 - z2*y1
    y = w2*y1 - x2*z1 + y2*w1 + z2*x1
    z = w2*z1 + x2*y1 - y2*x1 + z2*w1
    
    return (w, x, y, z)


In [17]:
import numpy as np
from scipy.spatial.transform import Rotation as R

transform = "0.99925953721395411 0.011029544981853946 0.036860906451799354 -0.01509804 0.0055656347488827985 -0.98939137035587255 0.145167971588677 -0.025274920999999999 0.038070999419467191 -0.14485532576616622 -0.98872014928410379 0.55663189999999996 0 0 0 1"
transform_list = transform.split(" ")
camera_transform = []
for t in transform_list:
    camera_transform.append(float(t))

camera_transform_matrix = np.array(camera_transform).reshape(4, 4)
rotation_matrix = camera_transform_matrix[:3, :3]
translation_vector = camera_transform_matrix[:3, 3]

# Convert rotation matrix to quaternion
rotation = R.from_matrix(rotation_matrix)
quaternion = rotation.as_quat()

qx, qy, qz, qw = quaternion

# Print the results
print(f"Quaternion (QW, QX, QY, QZ): ({qw}, {qx}, {qy}, {qz})")
print(f"Translation Vector (T): {translation_vector}")

Quaternion (QW, QX, QY, QZ): (-0.07271178992085355, 0.9971673702109805, 0.004160580316428108, 0.018786190791474776)
Translation Vector (T): [-0.01509804 -0.02527492  0.5566319 ]


In [16]:
Q1 = [qw, qx, qy, qz]

v0 = (0,1,0)
# Q1 = (0.201833, 0.894234, 0.096727, -0.387625)
v1 = rotate_vector_by_quaternion(v0, Q1)
# print(v1)

V_perpendicular = find_perpendicular_vector(v1)
Q2 = quaternion_180_degree_rotation(V_perpendicular)
v2 = rotate_vector_by_quaternion(v1, Q2)
# print(v2)

# V_initial = np.array(v1)  # Replace with your vector (a, b, c)
# V_target = -V_initial  # The opposite vector (-a, -b, -c)
# Q2 = find_rotation_quaternion(V_initial, V_target)

Q3 = quaternion_multiply(Q1, Q2)

v3 = rotate_vector_by_quaternion(v0, Q3)
# print(v3)
QQ = ""
for i in range(4):
    QQ = QQ + str(round(Q3[i],6))
    if i < 3:
        QQ = QQ + " "
print(QQ)

-0.997151 -0.072498 -0.019595 -0.006956
